In [16]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

fatal: destination path 'hw1' already exists and is not an empty directory.


In [17]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()



  +---+
      |
      |
      |
     ===

Word: _ _ _ _ _
Tried letters: 
Game over! The word was:  HEARD


0

In [18]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman

    #Define all the letters we can choose from
    allLetters = set('abcdefghijklmnopqrstuvwxyz')

    #Make sure the letters are all undercase and then take them away from
    #[allLetters]
    remainingLetters = list(allLetters - {letter.lower() for letter in letters_tried})

    #Use random to pick one of the remainding letterd
    return random.choice(remainingLetters) if remainingLetters else ''

hangman_game(random_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: _ _ _ U _
Tried letters: G, M, N, O, Q, R, U
Game over! The word was:  VALUE


0

In [19]:
def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Bayesian inference method for playing hangman. The parameters given are as follows:

    - letters_tried (set[str]): A set of strings which consist of all the letters that have already
        been tried. For example, if 'A', 'E' has been guessed, `letters_tried = {'A', 'E'}`
    - word_pattern (list[str]): A list of single characters that describe the current guess state.
        For example, if the hangman state is _AB__, `word_pattern = ['_', 'A', 'B', '_', '_']`
    - word_counts (dict[str, int]): The word counts dictionary which contains all possible 5 letter
        words in our hangman game and their respective counts. For example, a key value pair could
        be 'AARON': 413.

    Return type: a single character string as your next best guess.
    '''
    # TODO: Implement inference code to play hangman optimally

    #Normalize the word coutns
    totalWordCount = sum(word_counts.values())

    #Compute P(W | E)
    posterior = {}
    for word, count in word_counts.items():

      #Check if the word is consistent with the current pattern
      if(all(word[i] == word_pattern[i] or word_pattern[i] == '_' for i in range(5))):
        posterior[word] = count / totalWordCount

    #-------------------------------------------------------------------------#

    #Normalize posterior probablities

    totalPosterior = sum(posterior.values())
    for word in posterior:
      posterior[word] /=  totalPosterior

    #-------------------------------------------------------------------------#

    #Compute P(L_i = l | E)

    letterProb = {chr(c): 0 for c in range(ord('a'), ord('z') + 1)}

    #Loop through the words that match the given pattern
    for word, prob in posterior.items():

      for i, letter in enumerate(word):

        #Only want to consider the unkown spots of the unguessed word
        if word_pattern[i] == '_' and letter.lower() not in letters_tried:
          #Sum the probabilities
          letterProb[letter.lower()] += prob

    #-------------------------------------------------------------------------#

    #Pick the best letter with the highest probability

    #Filter out letters that have already been tried before finding the max
    available_letters = {letter: prob for letter, prob in letterProb.items() if letter not in {l.lower() for l in letters_tried}}

    if available_letters:
        maxLetter = max(available_letters, key=available_letters.get)
    else:
        # Handle case where all letters have been tried
        maxLetter = ''

    return maxLetter

# Run the game
hangman_game(bayesian_inference)



  +---+
  O   |
      |
      |
     ===

Word: T H I R D
Tried letters: D, E, H, I, R, T
Congratulations! You guessed the word:  THIRD


1

In [20]:
from typing import Optional, Callable
# from tqdm import tqdm

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 1000,
    seed: int = 0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.

    #Set the random seed
    random.seed(seed)

    #Accuracy and fail count
    success = 0
    fail = 0

    #-------------------------------------------------------------------------#

    #Iterate though the number of benchmark runs

    for _ in range(num_runs):
      try:
        #Play the game
        gameResult = hangman_game(inference, interactive = False, seed = seed)

        #Check if the game was won
        if gameResult == 1:

          success += 1

        else:

          fail += 1

      except Exception as e:
        #Count as a failure if the function throws an error
         fail += 1
         print(f"Error: {e}")

    #Calculate the accuracy
    accuracy = success  / num_runs
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Successes: {success}")
    print(f"Fails: {fail}")

print("Random interference")
benchmark(random_inference)
print()
print("Bayesian interference")
benchmark(bayesian_inference)


Random interference
Accuracy: 0.00%
Successes: 0
Fails: 1000

Bayesian interference
Accuracy: 100.00%
Successes: 1000
Fails: 0


In [21]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
texlive is already the newest version (2021.20220204-1).
texlive-latex-extra is already the newest version (2021.20220204-1).
texlive-xetex is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [25]:
!jupyter nbconvert --to pdf cse150a_hangman.ipynb
from google.colab import files
files.download('cse150a_hangman.pdf')

[NbConvertApp] Converting notebook cse150a_hangman.ipynb to pdf
[NbConvertApp] Writing 25971 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 28405 bytes to cse150a_hangman.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>